In [ ]:
# import libraries 
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from mlmvn.layers import OutputLayer, cmplx_phase_activation
from mlmvn.loss import ComplexMSELoss

torch.manual_seed(0)  #  for repeatable results

<torch._C.Generator>

In [ ]:
# create data
x = torch.Tensor([[1., 1.],
               [1., -1.],
               [-1., 1.],
               [-1., -1.]])

x = x.type(torch.cdouble)

y = torch.Tensor([0., 1., 1., 0.]).reshape(x.shape[0], 1)


In [ ]:
class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = OutputLayer(2, 2)
        self.linear1 = OutputLayer(2, 1)
        self.phase_act = cmplx_phase_activation()

    def forward(self, x):
        x = self.linear(x)
        x = self.linear1(x)
        x = self.phase_act(x)
        return x

In [ ]:
model = BasicModel()
criterion = ComplexMSELoss.apply
optimizer = torch.optim.SGD(model.parameters(), lr=1)
categories =  2
periodicity = 2


In [ ]:
for t in range(20):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    loss = criterion(y_pred.view(-1), y, categories, periodicity)
    if t % 100 == 99:
        print(t, torch.abs(loss))

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
predictions = model(x)

In [ ]:
def angle2class(x: torch.tensor, categories, periodicity) -> torch.tensor:
    tmp = x.angle() + 2 * np.pi
    angle = torch.remainder(tmp, 2 * np.pi)

    # This will be the discrete output (the number of sector)
    o = torch.floor(categories * periodicity * angle / (2 * np.pi))
    return torch.remainder(o, categories)

angle2class(predictions, 2, 2)

tensor([[0.],
        [1.],
        [1.],
        [0.]], dtype=torch.float64, grad_fn=<RemainderBackward0>)